In [1]:
!nvidia-smi

No devices were found


In [2]:
#%pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12
#%pip install cucim-cu12 cupy-cuda12x

In [3]:
%pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
#%load_ext cudf.pandas

# To desable GPU usage
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

#from cucim.skimage.exposure import rescale_intensity
import tensorflow as tf
#import cupy as cp
#import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split
import gc

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # memory limit 16GB (16 * 1024 MB = 16384 MB) 
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=16384)] 
        )
        print("Set GPU memory limit to 16GB.")
    except RuntimeError as e:
        print("Error setting memory limit:", e)
else:
    print("No GPUs available.")

#print("Is torch using cuda? ",torch.cuda.is_available())
print("Is tensorflow using cuda? ",tf.test.is_built_with_cuda())
print("Is pandas using cuda? ",pd)


2024-11-30 03:19:45.897682: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 03:19:45.928979: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 03:19:45.938601: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 03:19:45.963108: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


No GPUs available.
Is tensorflow using cuda?  True
Is pandas using cuda?  <module 'pandas' from '/usr/local/lib/python3.11/dist-packages/pandas/__init__.py'>


2024-11-30 03:19:49.487163: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


In [5]:
name_mapping = [
    "box",
    "circularTorus",
    "cone",
    "coneOffset",
    "cylinder",
    "cylinderSlope",
    "dish",
    "mesh",
    "pyramid",
    "rectangularTorus",
    "sphere"
]

In [15]:
!ls /home/workspace/geometry-classifier/data

big_parquet.parquet  no-stratTF1		    tfr_strat50-1min
model1.keras	     photos_v3_parquet		    tfrecord
model4.keras	     tfr_strat100-2min		    tfrecord2
no-strat	     tfr_strat100-2min-notfiltered  training


In [29]:
base_path = '/home/workspace/geometry-classifier/data/'

import os

import tensorflow as tf

def parse_tfrecord(example_proto):
    """
    Parses a serialized TFRecord example and extracts image and class label.
    """
    feature_description = {
        'name': tf.io.FixedLenFeature([], tf.string),
        'image': tf.io.FixedLenFeature([], tf.string)
    }
    parsed_example = tf.io.parse_single_example(example_proto, feature_description)
    name = tf.io.decode_raw(parsed_example['name'], tf.float32)
    image = tf.io.decode_raw(parsed_example['image'], tf.float32)
    image = tf.reshape(image, (224, 224, 1))  # Reshape image data to 224x224x1
    class_label = tf.cast(name[0], tf.int32)  # Use the first value in 'name' as a class label
    return image, class_label

def serialize_example(image, class_label):
    """
    Serializes an image and class label into a TFRecord example.
    """
    feature = {
        'name': tf.train.Feature(bytes_list=tf.train.BytesList(value=[class_label.numpy().tobytes()])),
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image.numpy().tobytes()]))
    }
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example.SerializeToString()

def filter_tfrecord(input_path, output_path):
    """
    Reads a TFRecord file, filters out rows with a class label of -1,
    and writes the remaining rows to a new TFRecord file.
    """
    # Create a TFRecord writer for the output file
    with tf.io.TFRecordWriter(output_path, options="GZIP") as writer:
        raw_dataset = tf.data.TFRecordDataset(input_path, compression_type="GZIP")
        for raw_record in raw_dataset:
            image, class_label = parse_tfrecord(raw_record)
            if class_label != -1:  # Filter out class -1
                serialized_example = serialize_example(tf.convert_to_tensor(image), tf.convert_to_tensor(class_label))
                writer.write(serialized_example)

def process_tfrecord_folder(input_folder, output_folder):
    """
    Processes all `.tfrecord.gz` files in a folder, filtering out rows with a class label of -1.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Use glob to find all `.tfrecord.gz` files in the folder
    input_files = glob.glob(os.path.join(input_folder, "*.tfrecord.gz"))

    for input_path in input_files:
        file_name = os.path.basename(input_path)
        output_path = os.path.join(output_folder, file_name)

        print(f"Processing {input_path} -> {output_path}")
        filter_tfrecord(input_path, output_path)



process_tfrecord_folder(base_path+"tfr_strat100-2min-notfiltered", base_path+"tfr_strat100-2min")

#glob.glob(base_path+"tfr_strat100-2min-notfiltered/*.tfrecord.gz")

Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/train462.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/train462.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/train153.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/train153.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/test54.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/test54.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/train314.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/train314.tfrecord.gz


2024-11-30 03:33:24.867141: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/train374.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/train374.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/train259.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/train259.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/test17.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/test17.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/train204.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/train204.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/train68.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/train68.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfil

2024-11-30 03:33:54.809064: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/train43.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/train43.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/train223.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/train223.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/val18.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/val18.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/train381.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/train381.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/test8.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/test8.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/tr

2024-11-30 03:34:50.756751: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/train101.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/train101.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/train143.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/train143.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/train28.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/train28.tfrecord.gz
Processing /home/workspace/geometry-classifier/data/tfr_strat100-2min-notfiltered/test98.tfrecord.gz -> /home/workspace/geometry-classifier/data/tfr_strat100-2min/test98.tfrecord.gz


KeyboardInterrupt: 